## Search and read CMIP6 data by using xarray

In [1]:
# load useful packages
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

### open CMIP6 online catalog
### data from CMIP6 of DMS for fully coupled model simulations (ocean + atmosphere)

In [4]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
cat = col.search(variable_id=['dmso'])
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CNRM-CERFACS,CNRM-ESM2-1,1pctCO2,r1i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20181018
1,CMIP,CNRM-CERFACS,CNRM-ESM2-1,1pctCO2,r2i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20181031
2,CMIP,CNRM-CERFACS,CNRM-ESM2-1,1pctCO2,r3i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20181107
3,CMIP,CNRM-CERFACS,CNRM-ESM2-1,piControl,r1i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20181115
4,CMIP,CNRM-CERFACS,CNRM-ESM2-1,historical,r1i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20181206
5,CMIP,CNRM-CERFACS,CNRM-ESM2-1,1pctCO2,r4i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1...,NaN,20190328
6,ScenarioMIP,CNRM-CERFACS,CNRM-ESM2-1,ssp370,r1i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,NaN,20191021
7,ScenarioMIP,CNRM-CERFACS,CNRM-ESM2-1,ssp585,r1i1p1f2,Omon,dmso,gn,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,NaN,20191021


### data from CMIP6 of DMS for OMIP model simulations, where the atmosphere is prescribed 

In [7]:
import xarray as xr
import s3fs
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))
# list avilable data in the bucket
s3.ls('escience2022/Ada/monthly')
dir = 'escience2022/Ada/monthly'
files = [file for file in s3.ls(dir) if 'dms' in file ]
files

['escience2022/Ada/monthly/dmsos_Omon_CNRM-ESM2-1_historical_r1i1p1f2_gn_185001-201412.nc',
 'escience2022/Ada/monthly/dmsos_Omon_CNRM-ESM2-1_omip1_r1i1p1f2_gn_170001-194912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_CNRM-ESM2-1_omip1_r1i1p1f2_gn_195001-200912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_185001-185912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_186001-186912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_187001-187912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_188001-188912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_189001-189912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_190001-190912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_191001-191912.nc',
 'escience2022/Ada/monthly/dmsos_Omon_NorESM2-LM_esm-hist_r1i1p1f1_gn_192001-192912.nc',
 'escience2022/Ada/mon